In [1]:
import os
os.getcwd()

'd:\\Machine Learning\\Deep Learning\\DeepClassifier\\CNN_Classifier\\research'

In [2]:
os.chdir('../')

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_eval_data: Path
    params_image_size: list
    params_batch_size: int
    mlflow_uri: str
    all_params: dict
    

In [ ]:
from deepclassifier.utils import *
from deepclassifier.constants import *

In [ ]:
import os


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self) -> EvaluationConfig:

        eval_config = EvaluationConfig(path_of_model = self.config.training.trained_model_path,
                                       training_eval_data = self.config.evaluation.train_eval_data,
                                       params_image_size = self.params.IMAGE_SIZE,
                                       params_batch_size = self.params.BATCH_SIZE,
                                       mlflow_uri="https://dagshub.com/Manjunath-Bhashyam/CNN_Classifier.mlflow",
                                       all_params=self.params
                                                            )
        
        return eval_config

In [ ]:
eval_config = ConfigurationManager().get_validation_config()

In [ ]:
eval_config

In [4]:
import tensorflow as tf
model = tf.keras.models.load_model(r'artifacts\training\model.keras')

In [ ]:
import tensorflow as tf
import time
from deepclassifier.utils import save_json
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
dagshub.init(repo_owner='Manjunath-Bhashyam', repo_name='CNN_Classifier', mlflow=True)



class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_eval_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            mlflow.log_params(self.config.all_params)
        # Model registry does not work with file store
        if tracking_url_type_store != "file":

        #     # Register the model
        #     # There are other ways to use the Model Registry, which depends on the use case,
        #     # please refer to the doc for more information:
        #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
        else:
            mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

In [ ]:
import os

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Manjunath-Bhashyam/CNN_Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Manjunath-Bhashyam"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1653d28fb7c0e991dffd2682d9a7ff82eb483df5"

In [ ]:
pip install dagshub

In [ ]:
# mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 0.0.0.0 -p 1234